In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sns

# Parameters
snapshot_date = pd.to_datetime('2025-09-13')
input_path = 'D:/Lumen Hackathon/LUMEN HACKATHON PRESENT/Project/ml_service/data/raw/SubscriptionUseCase_Dataset.xlsx'
output_dir = 'D:/Lumen Hackathon/LUMEN HACKATHON PRESENT/Project/ml_service/data/processed'
os.makedirs(output_dir, exist_ok=True)

In [2]:
# Load data from all sheets
User_data = pd.read_excel(input_path, sheet_name='User_Data')
Subscriptions = pd.read_excel(input_path, sheet_name='Subscriptions')
Subscriptions_plans = pd.read_excel(input_path, sheet_name='Subscription_Plans')
Subscriptions_logs = pd.read_excel(input_path, sheet_name='Subscription_Logs')
Billing_information = pd.read_excel(input_path, sheet_name='Billing_Information')

In [3]:
# Merge the dataframes
merged_df = pd.merge(User_data, Subscriptions, on='User Id', how='inner')
merged_df = pd.merge(merged_df, Subscriptions_plans, on='Product Id', how='inner')
merged_df = pd.merge(merged_df, Billing_information, left_on='Subscription Id', right_on='subscription_id', how='inner')
merged_df = pd.merge(merged_df, Subscriptions_logs, left_on='Subscription Id', right_on='Subscription id', how='inner')

df = merged_df.copy()
df_orig = df.copy()
print("Shape:", df.shape)
df.head(5)

Shape: (86, 30)


,User Id,Name_x,Phone,Email,Status_x,Subscription Id,Subscription Type,Product Id,Status_y,Start Date,...,amount,billing_date,payment_status,Unnamed: 5,Unnamed: 6,Subscription id,current status,next status,action,action date
0,7,User7,1234567807,user7@example.com,inactive,43,yearly,91,PAUSED,2024-02-12,...,228.58,2024-11-18,failed,NaN,NaN,43,initialized,active,renew_failed,2024-10-30
1,11,User11,1234567811,user11@example.com,active,54,monthly,20,PAUSED,2024-02-19,...,308.56,2024-01-04,paid,NaN,NaN,54,initialized,paused,renew_failed,2024-09-23
2,13,User13,1234567813,user13@example.com,inactive,75,yearly,22,active,2024-10-01,...,335.56,2024-07-14,pending,NaN,NaN,75,active,active,renew_failed,2024-07-17
3,13,User13,1234567813,user13@example.com,inactive,75,yearly,22,active,2024-10-01,...,335.56,2024-07-14,pending,NaN,NaN,75,initialized,paused,renew,2023-12-26
4,13,User13,1234567813,user13@example.com,inactive,75,yearly,22,active,2024-10-01,...,335.56,2024-07-14,pending,NaN,NaN,75,initialized,paused,renew,2024-08-28


In [4]:
# Helper: find similar column names
def find_cols(df, candidates):
    found = []
    for c in candidates:
        matches = [col for col in df.columns if c.lower() in col.lower()]
        found.extend(matches)
    return list(dict.fromkeys(found))

date_candidates = ['start_date','signup_date','created_at','subscription_start']
renew_candidates = ['last_renewal_date','renewal_date','last_renewed','renewed_at']
cancel_candidates = ['cancellation_date','cancel_date','cancelled_at','date_cancelled', 'terminated date']
auto_candidates = ['auto_renew','autorenew','auto_renewal','auto_renewed','renewal_mode']

print("Start-like cols found:", find_cols(df, date_candidates))
print("Renewal-like cols found:", find_cols(df, renew_candidates))
print("Cancel-like cols found:", find_cols(df, cancel_candidates))
print("Auto-renew-like cols found:", find_cols(df, auto_candidates))

Start-like cols found: []
Renewal-like cols found: []
Cancel-like cols found: ['Terminated Date']
Auto-renew-like cols found: []


In [5]:
# Normalize date columns
start_col = 'Start Date'
renew_col = 'Last Renewed Date'
cancel_col = 'Terminated Date'
auto_col = 'Auto Renewal Allowed'

for c in [start_col, renew_col, cancel_col]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')

# record counts for report
report = {}
report['n_rows'] = len(df)
report['start_missing'] = int(df[start_col].isna().sum()) if start_col in df.columns else None
report['renewal_missing'] = int(df[renew_col].isna().sum()) if renew_col in df.columns else None
report['cancel_missing'] = int(df[cancel_col].isna().sum()) if cancel_col in df.columns else None

In [6]:
# Compute tenure_days robustly
df = df.copy()
end_date = None
if cancel_col in df.columns:
    end_date = df[cancel_col]
else:
    end_date = pd.Series([pd.NaT]*len(df))

if renew_col in df.columns:
    end_date = end_date.fillna(df[renew_col])

end_date = end_date.fillna(snapshot_date)

if start_col in df.columns:
    df['tenure_days'] = (end_date - df[start_col]).dt.days
else:
    df['tenure_days'] = np.nan

negatives = (df['tenure_days'] < 0).sum()
if negatives > 0:
    df.loc[df['tenure_days'] < 0, 'tenure_days'] = np.nan
report['tenure_negatives'] = int(negatives)

df['start_date_missing_flag'] = df[start_col].isna() if start_col in df.columns else True
report['tenure_missing'] = int(df['tenure_days'].isna().sum())

In [7]:
# Compute days_since_renewal
if renew_col in df.columns:
    df['days_since_renewal'] = (snapshot_date - df[renew_col]).dt.days
else:
    df['days_since_renewal'] = np.nan

if start_col in df.columns:
    mask = df['days_since_renewal'].isna() & df[start_col].notna()
    df.loc[mask, 'days_since_renewal'] = (snapshot_date - df.loc[mask, start_col]).dt.days

neg_days = (df['days_since_renewal'] < 0).sum()
if neg_days > 0:
    df.loc[df['days_since_renewal'] < 0, 'days_since_renewal'] = np.nan
report['days_since_renewal_missing'] = int(df['days_since_renewal'].isna().sum())
report['days_since_renewal_negatives'] = int(neg_days)

In [8]:
# Parse is_auto_renew robustly
def parse_bool_auto(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in ('1','true','yes','y','t','on'):
        return 1
    if s in ('0','false','no','n','f','off'):
        return 0
    try:
        fv = float(s)
        if fv == 1.0:
            return 1
        if fv == 0.0:
            return 0
    except:
        pass
    if s and s[0] in ('y','n','t','f','1','0'):
        return 1 if s[0] in ('y','t','1') else 0
    return np.nan

if auto_col in df.columns:
    df['is_auto_renew'] = df[auto_col].apply(parse_bool_auto)
else:
    df['is_auto_renew'] = np.nan

df['is_auto_renew_missing_flag'] = df['is_auto_renew'].isna().astype(int)
impute_auto_to_zero = False
if impute_auto_to_zero:
    df['is_auto_renew_imputed'] = df['is_auto_renew'].isna().astype(int)
    df['is_auto_renew'] = df['is_auto_renew'].fillna(0)
else:
    df['is_auto_renew_imputed'] = 0
report['is_auto_renew_missing'] = int(df['is_auto_renew'].isna().sum())

In [9]:
# Basic diagnostics & plots
plt.figure(figsize=(8,5))
sns.histplot(df['tenure_days'].dropna(), bins=50, kde=False)
plt.title('tenure_days distribution')
plt.xlabel('tenure_days')
plt.savefig(os.path.join(output_dir, 'tenure_distribution.png'))
plt.close()

plt.figure(figsize=(8,5))
sns.histplot(df['days_since_renewal'].dropna(), bins=50, kde=False)
plt.title('days_since_renewal distribution')
plt.savefig(os.path.join(output_dir, 'days_since_renewal_distribution.png'))
plt.close()

In [10]:
# Transformations & bins
df['tenure_days_log1p'] = np.log1p(df['tenure_days'].clip(lower=0))
df['tenure_bin'] = pd.cut(df['tenure_days'], bins=[-1,7,30,90,180,365,np.inf], labels=['0-7','8-30','31-90','91-180','181-365','365+'])
try:
    df['tenure_qbin'] = pd.qcut(df['tenure_days'].rank(method='first'), q=5, labels=False)
except:
    df['tenure_qbin'] = np.nan

In [11]:
# Final save & write report
df.to_csv(os.path.join(output_dir, 'subscription_lifecycle_features.csv'), index=False)
import json
report['files_generated'] = {
    'processed_csv': os.path.join(output_dir,'subscription_lifecycle_features.csv'),
    'plots': ['tenure_distribution.png', 'days_since_renewal_distribution.png']
}
with open(os.path.join(output_dir,'lifecycle_report.json'),'w') as f:
    json.dump(report, f, indent=2)

print("Done. Processed file and report saved to", output_dir)

Done. Processed file and report saved to D:/Lumen Hackathon/LUMEN HACKATHON PRESENT/Project/ml_service/data/processed


In [12]:
# QA checks
assert 'tenure_days' in df.columns, 'tenure_days missing'
assert 'days_since_renewal' in df.columns, 'days_since_renewal missing'
assert 'is_auto_renew' in df.columns, 'is_auto_renew missing'
assert (df['tenure_days'].dropna() >= 0).all(), 'Some tenure_days < 0 remain'
assert (df['days_since_renewal'].dropna() >= 0).all(), 'Some days_since_renewal < 0 remain'

missing_tenure_pct = df['tenure_days'].isna().mean()
missing_auto_pct = df['is_auto_renew'].isna().mean()
print(f"Percentage missing: tenure_days={missing_tenure_pct:.2%}, is_auto_renew={missing_auto_pct:.2%}")

if missing_tenure_pct > 0.10:
    df[df['tenure_days'].isna()].head(100).to_csv(os.path.join(output_dir,'qc_missing_tenure_sample.csv'), index=False)
if missing_auto_pct > 0.5:
    df[df['is_auto_renew'].isna()].head(100).to_csv(os.path.join(output_dir,'qc_missing_auto_sample.csv'), index=False)

Percentage missing: tenure_days=44.19%, is_auto_renew=0.00%
